# Duty Scheduler Instructions

## What are the steps to create a schedule


1.   Input your staff names and email
2.   Input their unavailable days
3.   Add start date and duration of the schedule
4.   Add days that you would like to skip
5.   Run the Program
6.   It will automatically download an ics file and a text file

ics files, also known as iCal files, are a calendar format that is widely used. With this file, you will be able to automatically import calendar events for each person autopopulated with their names, email, and what shift they are on that day.

Roompact does not have a way to do this, so if you do not want to use the ics file and copy over the days into roompact, you can just use the text file that it will also download. this contains data that looks like this

**date / primaryName / SecondaryName**

*   11/1 Christina Stauffer  Tiera Fields
*   11/2 Jared Belcher

with this you can more easily copy to roompact






# Import and Setup (Do Not Edit)


In [35]:
from datetime import date as d
from datetime import datetime as dt
import random
import sys
!{sys.executable} -m pip install icalendar
from icalendar import Calendar, Event, vCalAddress, vText
import uuid

In [36]:
random.seed(str(dt.now()))

In [37]:
sched_cal = Calendar()
sched_cal['summary'] = "Duty Schedule"
sched_cal.add('prodid', '-//My calendar product//mxm.dk//')

sched_cal.add('version', '2.0')

# Update People


In [38]:
people = [
    {"name": "Name1",
     "email": "n1@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-02'),
    ]},
    {"name": "Name2",
     "email": "n2@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-05'),
    ]},
    {"name": "Name3",
     "email": "n3@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-01'),
    ]},
    {"name": "Name4",
     "email": "n4@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-27'),
    ]},
    {"name": "Name5",
     "email": "n5@udel.edu",
     "p_wd": 0,
     "s_wd": 0,
     "p_we": 0,
     "s_we": 0,
     "unavail": [
        d.fromisoformat('2023-11-09'),
    ]},
]

# create schedule dates
Set the start_date variable like this d(year, month, day)
set amt_days_to_sched variable to the amount of days that you would like to schedule. Ie. 30 for November, 28 for February, etc

For Dates that you want to exclude from scheduling, please use the exclude_dates variable. add d(year,month,day) with a comma separating each day inside of the square brackets []

In [39]:
start_date = d(2023, 11, 1)
amt_days_to_sched = 30
exclude_dates = []

# Scheduler (Do Not Edit)


In [40]:
schedule = []
for date_of_month in range(1, amt_days_to_sched + 1):
    t_date = d(start_date.year, start_date.month, date_of_month)
    avail_people=[]
    for person in people:
        if t_date not in person['unavail']:
            avail_people.append(person)
    schedule.append({"date": t_date, "avail_people": avail_people,"prim": None, "sec": None, "is_weekday": True if d.weekday(t_date) < 3 or d.weekday(t_date) > 5 else False})

In [41]:
# function for sorting keys

def get_duty_count_by_p_wd(person):
    return person['p_wd']

def get_duty_count_by_p_we(person):
    return person['p_we']

def get_duty_count_by_s_wd(person):
    return person['s_wd']

def get_duty_count_by_s_we(person):
    return person['s_we']

In [42]:
# sort people in day by selected shift and daytype

def sort_people_in_day(date, prim):
    random.shuffle(date['avail_people'])
    if prim is True and date['is_weekday'] is True:
        return date['avail_people'].sort(key=get_duty_count_by_p_wd)
    elif prim is True and date['is_weekday'] is False:
        return date['avail_people'].sort(key=get_duty_count_by_p_we)
    elif prim is False and date['is_weekday'] is True:
        return date['avail_people'].sort(key=get_duty_count_by_s_wd)
    else:
        return date['avail_people'].sort(key=get_duty_count_by_s_we)

In [43]:
def assign_shift(date, prim):
    i_person = 0
    if prim is True:
        while date['sec'] == date['avail_people'][i_person]:
            if i_person is len(date['avail_people']):
                raise IndexError("Could not find person available that was not self")
            else:
                i_person+=1
        date['prim'] = date['avail_people'][i_person]
        shift_event = Event()
        shift_event['uid'] = uuid.uuid1()
        shift_event['summary'] = date['prim']['name'] + " - Primary"
        attendee = vCalAddress('MAILTO:'+date['prim']['email'])
        attendee.params['cn'] = vText(date['prim']['name'])
        attendee.params['ROLE'] = vText('REQ-PARTICIPANT')
        shift_event.add('attendee', attendee, encode=0)
        shift_event.add('dtstamp', dt.now())
        shift_event.add('dtstart', dt(date['date'].year, date['date'].month, date['date'].day, 19, 0,0))
        shift_event.add('dtend', dt(date['date'].year, date['date'].month, date['date'].day, 23, 59,59))
        shift_event['dtstamp'].to_ical()
        shift_event['dtstart'].to_ical()
        shift_event['dtend'].to_ical()
        sched_cal.add_component(shift_event)
        if date['is_weekday'] is True:
            date['avail_people'][i_person]['p_wd'] = date['avail_people'][i_person]['p_wd'] + 1
        else:
            date['avail_people'][i_person]['p_we'] = date['avail_people'][i_person]['p_we'] + 1
        # eventually add to calendar
    else:
        while date['prim'] == date['avail_people'][i_person]:
            if i_person is len(date['avail_people']):
                raise IndexError("Could not find person available that was not self")
            else:
                i_person+=1
        date['sec'] = date['avail_people'][i_person]
        shift_event = Event()
        shift_event['uid'] = uuid.uuid1()
        shift_event['summary'] = date['sec']['name'] + " - Secondary"
        attendee = vCalAddress('MAILTO:'+date['sec']['email'])
        attendee.params['cn'] = vText(date['sec']['name'])
        attendee.params['ROLE'] = vText('REQ-PARTICIPANT')
        shift_event.add('attendee', attendee, encode=0)
        shift_event.add('dtstamp', dt.now())
        shift_event.add('dtstart', dt(date['date'].year, date['date'].month, date['date'].day, 7, 0,0))
        shift_event.add('dtend', dt(date['date'].year, date['date'].month, date['date'].day, 23, 59,59))
        shift_event['dtstamp'].to_ical()
        shift_event['dtstart'].to_ical()
        shift_event['dtend'].to_ical()
        sched_cal.add_component(shift_event)
        if date['is_weekday'] is True:
            date['avail_people'][i_person]['s_wd'] = date['avail_people'][i_person]['s_wd'] + 1
        else:
            date['avail_people'][i_person]['s_we'] = date['avail_people'][i_person]['s_we'] + 1
        # eventually add to calendar

In [44]:
for day in schedule:
    sort_people_in_day(day, True)
    assign_shift(day, True)
    sort_people_in_day(day, False)
    assign_shift(day, False)


# Output (Do Not Edit)

In [45]:
with open("schedule.ics", 'w') as cal_file:
    cal_file.write(sched_cal.to_ical().decode("utf-8"))

In [46]:
dty_string = ""
for day in schedule:
  dty_string = dty_string + str(day['date'].month)  + "/" + str(day['date'].day) + " " + day['prim']['name'] + " " + day['sec']['name'] + '\n'
dty_string

'11/1 Name2 Name5\n11/2 Name3 Name2\n11/3 Name4 Name1\n11/4 Name5 Name4\n11/5 Name3 Name4\n11/6 Name1 Name3\n11/7 Name4 Name1\n11/8 Name5 Name2\n11/9 Name2 Name3\n11/10 Name1 Name5\n11/11 Name3 Name2\n11/12 Name4 Name5\n11/13 Name3 Name2\n11/14 Name2 Name4\n11/15 Name1 Name3\n11/16 Name4 Name1\n11/17 Name1 Name3\n11/18 Name2 Name4\n11/19 Name5 Name1\n11/20 Name3 Name1\n11/21 Name2 Name5\n11/22 Name5 Name4\n11/23 Name5 Name3\n11/24 Name3 Name5\n11/25 Name5 Name2\n11/26 Name1 Name3\n11/27 Name5 Name2\n11/28 Name4 Name1\n11/29 Name1 Name5\n11/30 Name1 Name5\n'

In [47]:
with open("schedule.txt", 'w') as txt_file:
  txt_file.write(dty_string)